In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2021-07-31
# GitHub: https://github.com/jaaack-wang 
# About: Intro and word vectors for Stanford CS224N- NLP with Deep Learning | Winter 2019

# Table of Contents
- [1. Casual takeaways](#1)
- [2. WordNet](#2)
    - [2.1 Quick facts](#2-1)
    - [2.2 Accessing WordNet](#2-2)
    - [2.3 Limitations](#2-3)
- [3. Word vectors](#3)
    - [3.1 One-hot vectors](#3-1)
    - [3.2 Word vectors (embeddings)](#3-2)
    - [3.3 Word2vec algorithm basics](#3-3)
        * [3.3.1 Basic ideas](#3-3-1)
        * [3.3.2 Math behind](#3-3-2)
- [4. References](#4)
- [5. Appendix: gradients for loss function (2)](#5)

<a name='1'></a>
# 1. Casual takeaways

- Language isn't a formal system. Language is glorious chaos. Quoted from https://xkcd.com/1576/
- Commonest linguistic way of thinking of meanings: signifier (symbol) $\Longleftrightarrow$ signified (idea or thing) 
    - = denotational semantics. 
    - <font color='blue'>Note: <font color='black'>This idea has been poplularized by Ferdinand de Saussure in the his <i>Course in General Linguistics</i> where he, however, proposed that <ins>"language is a formal system of signs that express idea"</ins>. [More](https://en.wikipedia.org/wiki/Course_in_General_Linguistics).
- Ways of word meanings represented by computers:
    - [WordNet](#2): A traditional and manually labelled way.
    - [Words vectors](#3):  
        - [One-hot vectors](#3-1)
        - [Word vectors (embeddings)](3-2)
- NLTK is sort like "Swiss Army Knife of NLP", meaning that it's not terribly good for anything, but has a lot of basic tools. LOL.
- In traditional NLP, we regard words as discrete symbols: one-hot encoding until 2012. 
- “You shall know a word by the company it keeps” (J. R. Firth 1957: 11)

<a name='2'></a>
# 2. WordNet

Unless specifically stated, the term WordNet refers to the one designed for English language by Princeton 

<a name='2-1'></a>
## 2.1 Quick facts

- WordNet was first created in English only in the Cognitive Science Laboratory (Department of Psychology) of Princeton University starting in 1985, but it is currently housed in the Department of Computer Science.
- The most recent Windows version of WordNet is 2.1, released in March 2005. Version 3.0 for Unix/Linux/Solaris/etc. was released in December 2006. Version 3.1 is currently available only online.
- WordNet® is a large lexical database of semantic relations between words in English. It **includes the lexical categories nouns, verbs, adjectives and adverbs but ignores prepositions, determiners and other function words.**
- The main relation among words in WordNet is synonymy. **Synonyms--words that denote the same concept and are interchangeable in many contexts--are grouped into unordered sets (synsets).** Other semantic relations include: hyponyms, and meronyms etc.
- WordNet for other languages: https://en.wikipedia.org/wiki/WordNet#Other_languages 

<a name='2-2'></a>
## 2.2 Accessing WordNet
- Web interface: http://wordnetweb.princeton.edu/perl/webwn
- Official Downloads for associated packages and tools: https://wordnet.princeton.edu/download
- Python using [nltk](https://www.nltk.org)(Natural Language Toolkit). Below is example code: 
    - First, make sure that you have nltk installed. Run the command:  
```
pip install --user -U nltk
```
    - Second, make sure you have the WordNet dataset installed. In python/ipython, run:
```python
import nltk
nltk.download('wordnet')
```
    - Then, you can try the following code to utilize WordNet (A more detailed guide: http://www.nltk.org/howto/wordnet.html) 

In [2]:
# commmon way to import wordnet
from nltk.corpus import wordnet as wn


# look up a word using synsets(): a set of synonyms that share a common meaning.
# a synset is identified with a 3-part name of the form: word.pos.nn
print('Synsets for the word "invite" in WordNet:\n\n', wn.synsets('invite'))

Synsets for the word "invite" in WordNet:

 [Synset('invite.n.01'), Synset('invite.v.01'), Synset('invite.v.02'), Synset('tempt.v.03'), Synset('invite.v.04'), Synset('invite.v.05'), Synset('invite.v.06'), Synset('invite.v.07'), Synset('receive.v.05')]


In [3]:
# We can constrain the search by specifying the part of speech
# parts of speech available: ADJ, ADV, ADJ_SAT, NOUN, VERB
# ADJ_SAT: see https://stackoverflow.com/questions/18817396/what-part-of-speech-does-s-stand-for-in-wordnet-synsets

# Way one
print(f'{"-"*20}Way one{"-"*20}')
print('Synsets for the noun "invite" in WordNet:\n\n', wn.synsets('invite', pos=wn.NOUN))

# Way two
print(f'\n\n{"-"*20}Way two{"-"*20}')
# pos: {'n':'noun', 'v':'verb', 's':'adj (s)', 'a':'adj', 'r':'adv'}
print('Synsets for the noun "invite" in WordNet:\n\n', [s for s in wn.synsets('invite') if s.pos()=='n'])

--------------------Way one--------------------
Synsets for the noun "invite" in WordNet:

 [Synset('invite.n.01')]


--------------------Way two--------------------
Synsets for the noun "invite" in WordNet:

 [Synset('invite.n.01')]


In [4]:
# check definition of a synset
print(f'{"-"*20}Definition{"-"*20}')
print('The definition for invite as a noun:\n\n', wn.synset('invite.n.01').definition())

# check the related examples
print(f'\n\n{"-"*20}Examples{"-"*20}')
print('The definition for invite as a noun:\n\n', wn.synset('invite.n.01').examples())

# check the hypernyms
print(f'\n\n{"-"*20}Hypernyms{"-"*20}')
print('The hypernyms for invite as a noun:\n\n', wn.synset('invite.n.01').hypernyms())

--------------------Definition--------------------
The definition for invite as a noun:

 a colloquial expression for invitation


--------------------Examples--------------------
The definition for invite as a noun:

 ["he didn't get no invite to the party"]


--------------------Hypernyms--------------------
The hypernyms for invite as a noun:

 [Synset('invitation.n.01')]


In [5]:
# check all Synsets availble by wn.all_synsets()
# we can also specify the part of speech here

for idx, noun in enumerate(wn.all_synsets('n')):
    print(noun)
    if idx == 5:
        break

Synset('entity.n.01')
Synset('physical_entity.n.01')
Synset('abstraction.n.06')
Synset('thing.n.12')
Synset('object.n.01')
Synset('whole.n.02')


<a name='2-3'></a>
## 2.3 Limitations
- Great as a resource but missing nuance
• e.g., “proficient” is listed as a synonym for “good”
This is only correct in some contexts
- Missing new meanings of words
• e.g., wicked, badass, nifty, wizard, genius, ninja, bombest
- Impossible to keep up-to-date!
- Subjective
- **Requires human labor to create and adapt**
- Can’t compute accurate word similarity
    - nltk does provide a way to compute word similarity, but it is too simplistic and subjective: synset1.path_similarity(synset2): Return a score denoting how similar two word senses are, based on the shortest path that connects the senses in the is-a (hypernym/hypnoym) taxonomy. The score is in the range 0 to 1. See: http://www.nltk.org/howto/wordnet.html

In [6]:
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
print('The path similarity between cat(noun) and dog(noun): ', dog.path_similarity(cat))

The path similarity between cat(noun) and dog(noun):  0.2


<a name='3'></a>
# 3. Word vectors

My notes regarding word vectors and word2vec algorithms: https://github.com/jaaack-wang/dl-nlp-using-paddlenlp/tree/main/paddlenlp_updated_notes_English/WordEmbedding.

<a name='3-1'></a>
## 3.1 One-hot vectors
- Representing words as discrete symbols. 
- Popular in traditional NLP till 2012 where neural network trained word embeddings took place

**Problems**:
- The vocabulary size can be very large.
- Does not show any association between words as any two one-hot vectors are orthogonal. e.g., in web search, "seattle motel" != "seattle hotel".

**Solutions**:
- Could try to rely on WordNet’s list of synonyms to get similarity? • But it is well-known to fail badly: incompleteness, etc.
- Instead: learn to encode similarity in the vectors themselves.

<a name='3-2'></a>
## 3.2 Word vectors (embeddings)

**Basic ideas**:
- Distributional semantics: A word’s meaning is given by the words that frequently appear close-by.
- When a word w appears in a text, its context is the set of words that appear nearby (within a fixed-size window).
- Use the many contexts of w to build up a representation of w
- We will build a dense vector for each word, chosen so that it is similar to vectors of words that appear in similar contexts


<a name='3-3'></a>
## 3.3 Word2vec algorithm basics

Word2vec (Mikolov et al. 2013) is a framework for learning word vectors

<a name='3-3-1'></a>
### 3.3.1 Basic ideas 
- We have a large corpus (“body”) of text
- Every word in a fixed vocabulary is represented by a vector
- Go through each position t in the text, which has **a center word $c$** and **context (“outside”) words $o$**
- Use the similarity of the word vectors for $c$ and $o$ to calculate the probability of $o$ given $c$ (or vice versa)
- Keep adjusting the word vectors to maximize this probability

<img src='../images/1-word2vec-example.png' width='800' height='300'>

<a name='3-3-2'></a>
### 3.3.2 Math

**Objective/loss/cost function**

For each position $t = 1, ... , 𝑇$, predict context words within a window of fixed size $m$, given center word $w_j$. Data likelihood:

$$Likelihood = L(\theta) = \prod_{t}^{T} \prod_{\substack{-m \leq j \leq m \\ j \neq m}} P(w_{t+j}|{w_{t}; \theta}) \tag{1}$$

The objective/loss/cost function for $(1)$ is the average negative log likelihood: 

$$J(\theta) = - \frac{1}{T} logL(\theta) = - \frac{1}{T} \sum_{t=1}^{T} \sum_{\substack{-m \leq j \leq m \\ j \neq m}} log P(w_{t+j}|{w_{t}; \theta}) \tag{2}$$


**Prediction function**

Denote by $v_{c}$ and $v_{o}$ respectively the center word and the context word, using **softmax**, we get the following prediction function of predicting $v_{c}$ given $v_{o}$ and the vocubulary $V$:

$$P(o|c) = \frac{exp(\mathbf{u_{o}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \tag{3}$$

<br>
<br>

<img src='../images/1-softmax.png' width='800' height='300'>

**Gradient descent**

Knowledge needed: calculus: especially the chain rule, multivaraite calculus, partial derivative.   

In this lecture slide, Christopher Manning only introduced Batch Gradient Descent and Stochastic Gradient Descent, but not Mini-batch Gradient Descent (he did in the second lecture). Using Mini-batch Gradient Descent, we will update the parameters after looping through k examples (usually 16, 32, 64, $2^n$...). In a way, Batch Gradient Descent (update the parameters after looping through all the examples) and Stochastic Gradient Descent (update the parameters for every example encountered) are two extreme instances of Mini-batch Gradient Descent. 


<font color='red'><b>Check the details of how to derive the gradients for the loss function $(2)$ in the Appendix.</b></font>

<img src='../images/1-gradient-descent-1.png' width='800' height='300'>

<img src='../images/1-gradient-descent-2.png' width='800' height='300'>

<img src='../images/1-gradient-descent-3.png' width='800' height='300'>



<a name='4'></a>
# 4. References


- [Course website](http://web.stanford.edu/class/cs224n/index.html)

- [Lecture video](https://www.youtube.com/watch?v=8rXD5-xhemo&t=732s) 

- [Lecture slide](http://web.stanford.edu/class/cs224n/slides/cs224n-2021-lecture01-wordvecs1.pdf)

- [WordNet official website](https://wordnet.princeton.edu)

- [Wikipedia WordNet entry](https://en.wikipedia.org/wiki/WordNet#Other_languages)

- [NLTL WordNet Interface](http://www.nltk.org/howto/wordnet.html)




<a name='5'></a>
# 5. Appendix: gradients for loss function (2)


<font color='red'>Alternative using a different but neater set of notations: [word2vec gradients](https://courses.cs.ut.ee/MTAT.03.277/2015_fall/uploads/Main/word2vec.pdf)</font>

### <font color='blue'>Formula review</font>


$$J(\theta) = - \frac{1}{T} logL(\theta) = - \frac{1}{T} \sum_{t=1}^{T} \sum_{\substack{-m \leq j \leq m \\ j \neq m}} log P(w_{t+j}|{w_{t}; \theta}) \tag{2}$$

where $\sum_{t=1}^{T} \sum_{\substack{-m \leq j \leq m \\ j \neq m}} log P(w_{t+j}|{w_{t}; \theta})$ is equal to $log$ formula (3):

$$log P(o|c) = log \frac{exp(\mathbf{u_{o}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \tag{4}$$

### <font color='blue'>Some basic rules we need</font>
    
**Basic rules for logarithms**

- $log (\frac{a}{b}) = log(a) - log(b)$
- $log(e) = 1$ ($log$ here means $ln$)
- $log(a^b) = b \cdot log(a)$
- $log(e^x) = x$ ($log$ here means $ln$)


**Basic rules for derivatives**

- Addition and subtraction: $\frac{d}{dx} a(x) \pm b(x) = \frac{d}{dx} a(x) + \frac{d}{dx} b(x)$, e.g., $(x + x^2)' = (x)' + (x^2)' = 1 + 2x$


- Thus, constant and summation can be taken out front: $\frac{d}{dx} k \cdot a(x) = k \cdot a'(x)$, $\frac{d}{dx} \sum_{i=i}^{n} x_i = \sum_{i=i}^{n} \frac{d}{dx} x_i = \sum_{i=i}^{n} x_i' $


- Derivatives involving vectors: remember that $\mathbf{a^{T}b} = \sum_{i=1}^{n} a_i b_i$, suppose $a_i$ is a set of constants, thus $\frac{d}{db} \mathbf{a^{T}b} = \frac{d}{db} \sum_{i=1}^{n} a_i b_i = \sum_{i=1}^{n} a_i = \mathbf{a}$. In short, $\frac{d}{db} \mathbf{a^{T}b} = \mathbf{a}$. Similarly, this also holds: $\frac{d}{db} \mathbf{b^{T}a} = \mathbf{a}$


- Derivatives of exponential terms: $\frac{d}{dx} x^{k} = k \cdot x^{k-1}$, e.g., $(x^5)' = 5x^4$. Special case: $\frac{d}{dx} e^x = e^x$


- Product rule: $\frac{d}{dx} a(x) \cdot b(x) = b(x) \cdot \frac{d}{dx} a(x) + a(x) \cdot \frac{d}{dx} b(x) = b(x) \cdot a(x)' + a(x) \cdot b(x)'$, e.g., $(x \times x^2)' = x^2 \times 1 + x \times 2x = 3x^2 = (x^3)'$


- Chain rule: $\frac{d}{dx} a(b(x)) = \frac{d}{db} a(b(x)) \cdot \frac{d}{dx} b(x) = a'(b(x)) \cdot b'(x)$, e.g., suppose $a = n ^ 2$, $b = x + 1$, thus $a(b(x)) = (x+1)^2$. $\frac{d}{dx} a(b(x)) = a'(b(x)) \cdot b'(x) = 2(x+1) \times 1 = 2x+2 = (x^2 +2x + 1)'$


- Derivative of log: $\frac{d}{dx} log(x) = \frac{1}{log(x)}$


**Partial derivatives**

The idea of partial derivatives is to take the derivatives with regard to a variable and take the other variables, if any, as constant. For example, if we have functions $a$ and $b$, then the partial derivative with regard to function $a$ in the expresssion $ab$ is $b$, the first part of what is supposed to be under product rule. We use $\frac{\partial}{\partial a}$ to denote the partial derivative with regard to $a$.

### <font color='blue'> Let's crack the gradients for the loss function (2)!
    
Let's convert the loss function (2) using formula (4) and some logrithmic rules introduced above:
    
$$- \frac{1}{T} \log \frac{exp(\mathbf{u_{o}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} = - \frac{1}{T}(\log exp(\mathbf{u_{o}^{T} v_{c}}) - \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})) = - \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))$$


In short, we get:
    

$$J(\mathbf{v_{c} \;|\; u_{o}}) = - \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})) \tag{5}$$


### Partial derivative with regard to $v_c$
    
Let's derive the partial derivative with regard to $v_c$ (center word vector) first and then move on to $u_{o}$ (context word vector).

$$\frac{\partial}{\partial \mathbf{v_c}} (- \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))) = - \frac{1}{T} (\frac{\partial}{\partial \mathbf{v_c}}\mathbf{u_{o}^{T} v_{c}} -  \frac{\partial}{\partial \mathbf{v_c}}\log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))$$

Where:

$$\frac{\partial}{\partial \mathbf{v_c}}\mathbf{u_{o}^{T} v_{c}} = \mathbf{u_o} \tag{Step 1}$$

<br>

$$\frac{\partial}{\partial \mathbf{v_c}}\log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}) = \frac{1}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \cdot \sum_{x=1}^{V} \frac{\partial}{\partial \mathbf{v_c}}exp(\mathbf{u_{x}^{T} v_{c}})\tag{Step 2}$$

<br>
<br>
<font color='grey'>(Please note that, here we did a small trick by converting $\sum_{w \in V}$ into $\sum_{x=1}^{V}$ so that the partial derivative can be more easily computed.)</font>
<br>
Where


$$\frac{\partial}{\partial \mathbf{v_c}}exp(\mathbf{u_{x}^{T} v_{c}}) = exp(\mathbf{u_{x}^{T} v_{c}}) \cdot \frac{\partial}{\partial \mathbf{v_c}}\mathbf{u_{x}^{T} v_{c}} = exp(\mathbf{u_{x}^{T} v_{c}}) \cdot u_{x}\tag{Step 3}$$


Taking Step 2 and Step3 together, we get:

$$\frac{\partial}{\partial \mathbf{v_c}}\log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}) = \frac{\sum_{x=1}^{V}exp(\mathbf{u_{x}^{T} v_{c}}) \cdot u_{x}}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})}\tag{Step 4}$$

Step 4 can be rearranged as:

$$\sum_{x=1}^{V} \frac{exp(\mathbf{u_{x}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \cdot  u_{x} \tag{Step 5}$$

Recall the prediction function (3):

$$P(o|c) = \frac{exp(\mathbf{u_{o}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \tag{3}$$

We can rearrange Step 5 result accordingly:

$$\sum_{x=1}^{V} P(x|c) \cdot  u_{x} \tag{Step 6}$$

Taking everything together (Step 6 + Step 1), finally we get:

$$\frac{\partial}{\partial \mathbf{v_c}} (- \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))) = - \frac{1}{T}(\mathbf{u_o} - \sum_{x=1}^{V} P(x|c) \cdot  \mathbf{u_{x}})$$

<br><br>

### Partial derivative with regard to $u_o$

$$\frac{\partial}{\partial \mathbf{u_o}} (- \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))) = - \frac{1}{T} (\frac{\partial}{\partial \mathbf{u_o}}\mathbf{u_{o}^{T} v_{c}} -  \frac{\partial}{\partial \mathbf{u_o}}\log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))$$

Where:

$$\frac{\partial}{\partial \mathbf{u_o}}\mathbf{u_{o}^{T} v_{c}} = \mathbf{v_{c}} \tag{Step 1}$$

<br>

$$ \frac{\partial}{\partial \mathbf{u_o}}\log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}) = \frac{1}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \cdot \sum_{x=1}^{V} \frac{\partial}{\partial \mathbf{u_x}}exp(\mathbf{u_{x}^{T} v_{c}})\tag{Step 2}$$

Where:

$$\frac{\partial}{\partial \mathbf{u_x}}exp(\mathbf{u_{x}^{T} v_{c}}) = exp(\mathbf{u_{x}^{T} v_{c}}) \cdot  \mathbf{v_{c}}\tag{Step 3}$$

Taking Step 2 and Step3 together, we get:

$$\frac{\partial}{\partial \mathbf{u_o}}\mathbf{u_{o}^{T} v_{c}} = \frac{\sum_{x=1}^{V}exp(\mathbf{u_{x}^{T} v_{c}}) \cdot  \mathbf{v_{c}}}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \tag{Step 4}$$

Step 4 can be rearranged as:

$$\frac{\sum_{x=1}^{V}exp(\mathbf{u_{x}^{T} v_{c}}) \cdot  \mathbf{v_{c}}}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} = \sum_{x=1}^{V}\frac{exp(\mathbf{u_{x}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})}\mathbf{v_{c}} = \sum_{x=1}^{V} P(x|c) \cdot \mathbf{v_{c}}\tag{Step 5}$$


Finally, everything taken together:

$$\frac{\partial}{\partial \mathbf{u_o}} (- \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))) = - \frac{1}{T}(\mathbf{u_o} - \sum_{x=1}^{V} P(x|c) \cdot  \mathbf{v_{c}})$$